In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
import tqdm.notebook as tq
import solitaire_env_hard as solitaire_env_hard
import sys
import seaborn as sns
import pickle as pkl
import gp
import tqdm.notebook as tq
#sys.setrecursionlimit(10**6)

In [73]:
class Node:
    def __init__(self,state=None):
        self.visits = 0
        self.children = []
        self.state = state
        self.action_scores = np.array([np.inf for _ in range(6)])
        self.parent = None
        self.child = None
        self.hashmap = {}
        
        
        self.parent = None
        self.visitted = False
        
        self.childCount = []
        
        self.possible_actions = []
    def won(self):
        
        pile_len = len(self.state.pile)
        
        if pile_len !=0:
            return False
        
        for i in range(7):
            if len(self.state.tableau[i]) != 0:
                return False
            
        for i in range(4):
            if len(self.state.tableau[i]) != 13:
                return False
            
        return True
    
    def stuck(self):
        
        test_env = solitaire_env_hard.env()
        test_env.state = self.state
        test_env.state = test_env.copy_state()
        
        debug = [False]*6
        
        for action in range(6):
            w,taken = test_env.step(action,fp_flag=True,debug=debug)
            
            if taken == True:
                return False
            
            
        
        return True
    
    
    def find_possible_actions(self):
        
        possible_actions = []
        
        possible_tableau_to_foundation_reveal_actions = self.find_possible_tableau_to_foundation_reveal_actions()
        
        possible_pile_to_tableau_actions = self.find_possible_pile_to_tableau_actions()
        
        possible_to_foundation_stack_actions = self.find_possible_to_foundation_stack_actions()
        
        #possible_tableau_to_tableau_reveal_actions = self.find_possible_tableau_to_tableau_reveal_actions()
        
        #possible_tableau_to_tableau_not_reveal_actions = self.find_possible_tableau_to_tableau_not_reveal_actions()
        
        possible_foundation_to_tableau_actions = self.find_possible_foundation_to_tableau_actions()
        
        
        
        
        possible_actions.extend(possible_tableau_to_foundation_reveal_actions)
        
        possible_actions.extend(possible_pile_to_tableau_actions)
        
        possible_actions.extend(possible_to_foundation_stack_actions)
        
        possible_actions.extend(possible_foundation_to_tableau_actions)
        
        
        return possible_actions
      
    def find_possible_tableau_to_foundation_reveal_actions(self):
        
        moves = []
        
        for i in range(7):
            
            if len(self.state.tableau[i]) <2:
                continue
                
            if self.state.tableau[i][-2].face == 'up':
                continue
                
            cd = self.state.tableau[i][-1]
            
            #print(cd)
            suit = cd.get_suit_number()
            
            if cd.number == 1 or (len(self.state.foundation[suit])>0 and self.state.foundation[suit][-1].number == (cd.number-1)) :
                
                moves.append((0,i,cd,suit))
                
                
        return moves
    
    def find_possible_pile_to_tableau_actions(self):
        
        moves = []
        
        
        for i in range(len(self.state.pile)):
            
            if (i%3) == 0:
                cd1 = self.state.pile[i]
                
                for j in range(7):
                    
                    if len(self.state.tableau[j])>0:
                        
                        cd2 = self.state.tableau[j][-1]
                        
                        if cd1.color != cd2.color and cd1.number == (cd2.number-1):
                            moves.append((3,i,j))
                    
        return moves
    
    def default_policy_action(self):
        
        n = len(self.untried_actions)
        
        ind = random.sample(range(n),1)[0]
        
        action_index = self.untried_actions.pop(ind)
        
        return action_index, self.possible_actions[action_index]
    
    
    def best_action(self):
        
        ind = self.action_scores.argmax()
        
        
        return ind, self.possible_actions[ind]
    
    
    def add_child(self,action,new_node):
        
        self.children[action[0]].append(new_node)
        
    
    def random_child(self,action):
        
        child = random.sample(self.children[action[0]],1)[0]
        
        return child
    
    
        
    def find_possible_to_foundation_stack_actions(self):
        
        moves = []
        
        for i in range(len(self.state.pile)):
            
            if (i%3) == 0:
                
                cd1 = self.state.pile[i]
                
                suit = cd1.get_suit_number()
                
                if cd1.number == 1:
                    
                    moves.append((1,0,i))
                    
                elif len(self.state.foundation[suit])>=1:
                    
                    cd2 = self.state.foundation[suit][-1]
                    
                    if cd1.number == (cd2.number+1):
                    
                        moves.append((1,0,i))
                    
                    
        for i in range(7):
            
            if len(self.state.tableau[i])>0:
                
                cd1 = self.state.tableau[i][-1] 
                suit = cd1.get_suit_number()
                
                if cd1.number == 1:
                    
                    moves.append((1,1,i))
                    
                elif len(self.state.foundation[suit])>=1:
                    
                    cd2 = self.state.foundation[suit][-1]
                    
                    if cd1.number == (cd2.number+1):
                    
                        moves.append((1,1,i))
                    
                    
        return moves
    
    def find_possible_foundation_to_tableau_actions(self):
        
        moves = []
        
        for i in range(4):
            if len(self.state.foundation[i])>0:
                
                cd1 = self.state.foundation[i][-1]
                
                for j in range(7):
                    
                    if len(self.state.tableau[j])>0:
                        
                        cd2 = self.state.tableau[j][-1]
                        
                        if cd1.color != cd2.color and cd1.number == (cd2.number-1):
                            moves.append((4,cd1.get_suit_number(),j))
                            
                            
                            
        return moves
    def make_new_node(self):
        temp_en = solitaire_env_hard.env()
        
        temp_en.state = self.state
        
        temp_en.state = temp_en.copy_state()
        
        new_node = Node()
        
        new_node.state = temp_en.state
        
        return new_node
    def move_tableau_to_foundation_reveal(self,action):
        
        _, from_tableau, cd, suit_number = action
        
        
        new_node = self.make_new_node()
        
        new_node.state.tableau[from_tableau].pop()
        
        new_node.state.tableau[from_tableau][-1].face = 'up'
        new_node.state.foundation[suit_number].append(cd)
        
        return new_node
        
    def move_to_foundation_stack(self,action):
        
        _, typ, from_ind = action
        
        new_node = self.make_new_node()
        
        if typ == 0:
            
            cd = new_node.state.pile.pop(from_ind)
            
        elif typ == 1:
            
            cd = new_node.state.tableau[from_ind].pop()
            
            if len(new_node.state.tableau[from_ind]) >0:
                new_node.state.tableau[from_ind][-1].face = 'up'
        
        new_node.state.foundation[cd.get_suit_number()].append(cd)
        
        return new_node
        
    def move_pile_to_tableau(self,action):
        
        _,pile_position,to_tableau = action
        
        new_node = self.make_new_node()
        
        cd = new_node.state.pile.pop(pile_position)
        
        new_node.state.tableau[to_tableau].append(cd)
        
        return new_node
        
        
    def move_foundation_to_tableau(self,action):
        
        _, suit_number, to_tableau = action
        
        new_node = self.make_new_node()
        
        cd = new_node.pile.foundation[suit_number].pop()
 
        new_node.state.tableau[from_tableau].append(cd)
    
        return new_node
    
    def transition(self,action):
        
        action_type = action[0]
        next_node = None
        if action_type == 0:
            next_node = self.move_tableau_to_foundation_reveal(action)
            
        elif action_type == 1:
            next_node = self.move_to_foundation_stack(action)
        elif action_type == 2:
            next_node = self.move_tableau_to_tableau_reveal(action)
        elif action_type == 3:
            next_node = self.move_pile_to_tableau(action)
        elif action_type == 4:
            next_node = taken = self.move_foundation_to_tableau(action)
            
        elif action_type == 5:
            next_node = taken = self.move_tableau_to_tableau_not_reveal(action)
        
        return next_node

In [74]:
def update_scores(node,r):
        
        
    while node.parent is not None:

        parent = self.parent
        action = self.which_action_number

        parent.action_scores[action]+=r

        node = parent
            
            
            

In [75]:
number_of_trajectories = 100
sampling_width = 5

In [76]:
from tqdm import tqdm
env = solitaire_env_hard.env()
node0 = Node()
node0.state = env.state
counter = 0


for i in tqdm(range(number_of_trajectories)):
    
    node = node0
    
    print(node.visitted)
    won = node.won() 
    
    stuck = node.stuck()
    
    while won == False and stuck == False:
        
        counter +=1
        if node.visitted == False:
            
            possible_actions = node.find_possible_actions()
            
            print("number of possible actions",len(possible_actions))
            node.visitted = True
            
            node.possible_actions = possible_actions
            node.number_of_possible_actions = len(possible_actions)
            
            node.untried_actions = list(range(node.number_of_possible_actions))
            
            node.action_score = np.zeros(node.number_of_possible_actions)
            
            node.childCount = np.zeros(node.number_of_possible_actions)
        
            node.children = [[] for _ in range(node.number_of_possible_actions)]
        if len(node.untried_actions) == 0:
            
            action = node.best_action()
            
        else:
            action = node.default_policy_action()
            
            
        if node.childCount[action[0]] == sampling_width:
            new_node = node.random_child(action)
            
            
        else:
            next_node = node.transition(action[1])
            print(next_node)
            node.add_child(action,next_node)
            
            next_node.parent = node
            node.childCount[action[0]]+=1
            
            
        node = next_node
        
        
    
    update_scores(node,1 if won==True else 0)
        
        
print(counter)

  0%|          | 0/100 [00:00<?, ?it/s]

False
number of possible actions 3
number of possible actions 3
number of possible actions 2
number of possible actions 1
number of possible actions 1
number of possible actions 2
number of possible actions 1


AttributeError: 'Node' object has no attribute 'pile'